# Giới thiệu bài toán

# Giới thiệu bài toán
## 1. Đặt vấn đề: Giới hạn của biểu diễn one-hot

Trong xử lý ngôn ngữ tự nhiên (NLP), bước đầu tiên và quan trọng là **biểu diễn từ ngữ dưới dạng số** để đưa vào mô hình học máy. Một phương pháp trực quan và phổ biến là **vector one-hot**, trong đó mỗi từ được biểu diễn bằng một vector nhị phân có độ dài bằng kích thước từ điển, với đúng một phần tử bằng 1 và các phần tử còn lại bằng 0.

Mặc dù đơn giản và dễ triển khai, biểu diễn one-hot tồn tại những **hạn chế nghiêm trọng**:

- **Không có thông tin ngữ nghĩa**: Vector của “cat” và “dog” hoàn toàn trực giao, không phản ánh bất kỳ sự tương đồng nào giữa chúng.
- **Sparsity cao**: Độ dài vector bằng kích thước từ điển (có thể lên tới hàng triệu từ), dẫn đến tiêu tốn bộ nhớ và tính toán kém hiệu quả.
- **Không học được quan hệ ngữ cảnh**: Các vector không thể cập nhật để phản ánh mối quan hệ ngữ cảnh trong văn bản.

Vì vậy, cần có một phương pháp biểu diễn từ tốt hơn, vượt qua được các hạn chế trên


## 2. Giải pháp: Word Embedding

**Word embedding** là kỹ thuật học biểu diễn các từ dưới dạng vector thực (dense vector) có chiều thấp (ví dụ từ 50 đến 100, thấp hơn nhiều so với số từ khả dĩ trong từ điển của bất kỳ ngôn ngữ thông dụng nào), sao cho các vector này **mang thông tin ngữ nghĩa và ngữ cảnh**.

Khác với vector one-hot, các vector embedding được học từ dữ liệu, nhờ đó:
- Các từ có ý nghĩa gần nhau sẽ có vector gần nhau.
- Có thể thực hiện các phép toán hình học như:
  $$
  \text{vec}(\text{"king"}) - \text{vec}(\text{"man"}) + \text{vec}(\text{"woman"}) \approx \text{vec}(\text{"queen"})
  $$
- Được dùng như vector biểu diễn từ cho các bài toán xử lý ngôn ngữ tự nhiên khác như: phân loại văn bản, sinh văn bản, dịch máy, v.v.

Một trong những mô hình học embedding thành công và phổ biến nhất là **Word2Vec**, bao gồm hai biến thể chính: **Skip-Gram** và **CBOW**.


Word embedding không chỉ cải thiện khả năng xử lý ngôn ngữ của mô hình, mà còn:
- **Giảm đáng kể số chiều**
- **Tăng hiệu quả học**: Vector liên tục giúp mô hình tổng quát tốt hơn.
- **Cho phép học không giám sát**: Có thể huấn luyện embedding từ kho văn bản lớn mà không cần gán nhãn.


## 3. Ứng dụng của Word Embedding

Word embedding là nền tảng cốt lõi của hầu hết các mô hình NLP hiện đại. Một số ứng dụng tiêu biểu bao gồm:

- **Tìm kiếm ngữ nghĩa**: Gợi ý truy vấn hoặc tài liệu dựa trên từ tương đồng.
- **Phân loại văn bản**: Phân loại văn bản theo nhu cầu như phân loại email rác, phân loại xu hướng bình luận, ...
- **Dịch máy**: Embedding giúp mô hình hiểu nghĩa và ngữ pháp của từ.
- **Sinh văn bản**: Các mô hình như GPT sử dụng embedding làm lớp đầu vào


## 4. Mục tiêu của chương này

Trong chương này, chúng ta sẽ cùng tìm hiểu chi tiết mô hình **Skip-gram** và **CBOW**: cách xây dựng mô hình, các công thức toán học, và cách huấn luyện chúng

# Bài tập

## Câu 1:
Độ phức tạp tính toán của mỗi gradient là bao nhiêu? Nếu từ điển chứa một lượng lớn các từ, điều này sẽ gây ra vấn đề gì?

### 1.1. Độ phức tạp tính toán của mỗi gradient

Nhắc lại công thức tính gradient của hàm mất mát theo $\mathbf{v}_c$ trong mô hình Skip-gram (15.1.8):

$$
\frac{\partial \textrm{log}\, P(w_o \mid w_c)}{\partial \mathbf{v}_c} = \mathbf{u}_o - \sum_{j \in \mathcal{V}} P(w_j \mid w_c) \mathbf{u}_j
$$

Nhắc lại công thức tính gradient của hàm mất mát theo $\mathbf{v}_{o_i}$ trong mô hình CBOW (15.1.15):

$$
\frac{\partial \log\, P(w_c \mid \mathcal{W}_o)}{\partial \mathbf{v}_{o_i}} = \frac{1}{2m}\left(\mathbf{u}_c - \sum_{j \in \mathcal{V}} P(w_j \mid \mathcal{W}_o) \mathbf{u}_j \right)
$$


- Dễ thấy, đối với cả 2 mô hình, để tính được gradient ta phải tính xác suất có điều kiện cho mọi từ $w_j$ có trong từ điển $\mathcal{V}$ với một từ cho trước.
- Do đó, độ phức tạp tính toán của mỗi gradient trong cả 2 mô hình đều là $O(|\mathcal{V}| \cdot d)$, với $d$ là số chiều của vector embedding và $\mathcal{V}$ là từ điển


### 1.2. Vấn đề khi từ điển chứa một lượng lớn các từ

- Từ độ phức tạp trên ta thấy, mỗi lần huấn luyện cho một cặp từ, ta đều phải tính xác suất cho tất cả các từ trong từ điển
- Điều này rất tốn thời gian, dẫn đến huấn luyện rất chậm hội tụ, không khả thi với dữ liệu thực tế.

**Giải pháp:** Dùng kỹ thuật Negative Sampling (xem mục 15.2.1)

## Câu 2:
Có một số cụm từ cố định trong tiếng Anh bao gồm nhiều từ, chẳng hạn như “new york”. Bạn sẽ huấn luyện các vector từ của chúng như thế nào? Gợi ý: Xem phần 4 trong bài báo Word2vec[2].

### Ý tưởng: Nhận diện và tạo cụm từ (phrases) trước khi huấn luyện Word2Vec

Trước khi huấn luyện Word2Vec, ta xác định ra các cụm từ phổ biến trong một ngữ cảnh nhưng ít phổ biến trong các ngữ cảnh khác, như: `"new york"`, `"san francisco"`... (trong khi cụm `"this is"` thì xuất hiện trong rất nhiều ngữ cảnh), sau đó nối chúng lại và coi chúng là một từ riêng trong từ điển khi huấn luyện embedding.

---

## Phương pháp phát hiện cụm từ: Dựa trên thống kê đồng xuất hiện
Có nhiều phương pháp phát hiện cụm từ đã được nghiên cứu trước đó, nhưng chúng không nằm trong phạm vi nghiên cứu của bài báo nên nhóm tác giả chỉ sử dụng phương pháp đơn giản và đủ hiệu quả.

Ta kiểm tra xem cặp từ $w_i, w_j$ có nên được nối thành một cụm hay không bằng công thức sau:

$$
\text{score}(w_i, w_j) = \frac{C(w_i w_j) - \delta}{C(w_i) \cdot C(w_j)}
$$

Trong đó:
- $C(w_i)$, $C(w_j)$: tần suất của từng từ riêng lẻ,
- $C(w_i w_j)$: tần suất của cụm 2 từ xuất hiện liên tiếp,
- $\delta$: một hằng số để tránh nối những cụm có tần suất thấp (thường là 5).

Đồng thời đặt ra một ngưỡng cố định. Nếu score vượt ngưỡng này, ta xem $w_i w_j$ là cụm cố định.

> **Ví dụ 1**:  
> Nếu `"new"` xuất hiện 5000 lần, `"york"` 3000 lần, `"new york"` xuất hiện 2800 lần, thì:
>
> $\text{score}(\text{"new"}, \text{"york"}) = \frac{2800 - 5}{5000 \cdot 3000} \approx 0.000186$
> 
> Nếu ngưỡng là 0.0001 → `"new york"` được giữ lại thành `"new_york"`.

> **Ví dụ 2**:  
> Nếu `"this"` xuất hiện 10000 lần, `"is"` 9500 lần, `"this is"` xuất hiện 5000 lần, thì:
>
> $\text{score}(\text{"this"}, \text{"is"}) = \frac{5000 - 5}{10000 \cdot 9500} \approx 0.0000525789474$
> 
> Nếu ngưỡng là 0.0001 → `"this is"` không được nối lại thành cụm từ

## Câu 3:
Sử dụng mô hình skip-gam làm ví dụ để tìm hiểu về thiết kế của mô hình word2vec. Mối quan hệ giữa tích vô hướng của hai vector từ và độ tương tự cô-sin trong mô hình skip-gam là gì? Đối với một cặp từ có ngữ nghĩa gần nhau, tại sao hai vector từ này lại thường có độ tương tự cô-sin cao?

### 3.1. Mối quan hệ giữa tích vô hướng của hai vector từ và độ tương tự cô-sin

Nhắc lại công thức tính vô hướng trong trường hợp 2 vector từ:

$$
\boldsymbol{u}_{w_o}^\top \boldsymbol{v}_{w_c} = \|\boldsymbol{u}_{w_o}\| \cdot \|\boldsymbol{v}_{w_c}\| \cdot \cos(\theta)
$$

Trong đó:
- $\theta$: góc giữa hai vector.
- $\cos(\theta)$: **độ tương tự cosine** giữa hai vector, có miền giá trị từ -1 (khi 2 vector ngược hướng) đến 1 (khi 2 vector cùng hướng) và bằng 0 khi 2 vector vuông góc

👉 **Tích vô hướng càng lớn (theo chiều dương)**, nghĩa là:
- Hai vector càng **cùng hướng** (cosine gần 1),
- Hoặc có **độ dài lớn** (cũng ảnh hưởng, nhưng không mang nghĩa ngữ nghĩa nhiều bằng góc).



### 3.2. Vì sao các từ có nghĩa gần nhau thường có độ tương tự cosine cao?


Nhắc lại công thức của Skip-gram: Với cặp từ $(w_c, w_o)$, ta tối đa hóa xác suất sau:

$$
P(w_o | w_c) = \frac{\exp(\boldsymbol{u}_{w_o}^\top \boldsymbol{v}_{w_c})}{\sum_{w \in V} \exp(\boldsymbol{u}_w^\top \boldsymbol{v}_{w_c})}
$$

Quan sát công thức này ta thấy: Nếu hai từ hay cùng xuất hiện, tức là xác suất $P$ ở trên lớn, thì tích vô hướng $\boldsymbol{u}_{w_o}^\top \boldsymbol{v}_{w_c}$ cũng lớn.

Mặt khác, ta huấn luyện Skip-gram bằng cách cực đại hoá hợp lý xác suất $P$ đó. Do vậy, mô hình đã được huấn luyện sao cho làm tăng tích vô hướng của 2 vector từ tương tự nhau, từ đó làm tăng độ tương tự cosine.


**Tóm lại:**
- Trong mô hình Skip-Gram của Word2Vec, xác suất dự đoán từ phụ thuộc vào tích vô hướng giữa hai vector từ, mà tích vô hướng lại phụ thuộc trực tiếp vào độ tương tự cosine.
- Khi các từ có nghĩa gần nhau, chúng thường xuất hiện trong những ngữ cảnh giống nhau, khiến các vector của chúng hướng gần nhau trong không gian → độ tương tự cosine cao.